In [4]:
!pip freeze | grep requests

requests==2.32.3
requests-toolbelt==1.0.0


In [5]:
BASE = "http://localhost:5055"

SOFTWARE_LIST= "/api/softwares"
SOFTWARE_VERSIONS = "/get_versions"
SOFTWARE_DEPENDENCIES="/api/dependencies" #Params: name, version
SOFTWARE_SBOM="/api/sbom" #POST: name, version
SOFTWARE_VULNERABILITES="/api/vulnerabilities"


In [ ]:
import requests as r

name = "aws-sdk-cpp"
version = "1.11.117"

list = r.get(BASE+SOFTWARE_LIST)
print(list.text)
versions=r.get(f"{BASE}{SOFTWARE_VERSIONS}?name={name}")
print(versions.text)
dependencies=r.get(f"{BASE}{SOFTWARE_DEPENDENCIES}?name={name}&version={version}")
print(dependencies.text)
vulns = r.get(f"{BASE}{SOFTWARE_VULNERABILITES}?name={name}&version={version}")
print(vulns.text)
sbom = r.post(BASE+SOFTWARE_SBOM, json= {
    "name": name, "version": version
})
print(sbom.text)

In [8]:
import requests as r
list = r.get(BASE+SOFTWARE_LIST)

versions=r.get(f"{BASE}{SOFTWARE_VERSIONS}?name={name}")

dependencies=r.get(f"{BASE}{SOFTWARE_DEPENDENCIES}?name={name}&version={version}")

vulns = r.get(f"{BASE}{SOFTWARE_VULNERABILITES}?name={name}&version={version}")

sbom = r.post(BASE+SOFTWARE_SBOM, json= {
    "name": name, "version": version
})


NameError: name 'name' is not defined

In [19]:
%%capture
%pip install google-generativeai
%pip install -U instructor

In [9]:
# Setup lib
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
import google.generativeai as genai
from typing import Literal

from dotenv import load_dotenv
load_dotenv()
import os

# Define User intent enum
class ClassificationResponse(BaseModel):
    """
    A few-shot example of text classification:

    Examples:
    - "Get me the list of software!": SOFTWARE_LIST
    - "What are the versions for this software": SOFTWARE_VERSIONS
    - "What are libraries are used for aws-sdk": SOFTWARE_DEPENDENCIES
    - "List the vulnerabilities in linux?": SOFTWARE_VULNERABILITES
    - "Generate the SBOM for whisper": SOFTWARE_SBOM
    - "Write me a haiku": UNKNOWN
    """

    chain_of_thought: str = Field(
        ...,
        description="The chain of thought that led to the prediction.",
    )
    label: Literal["SOFTWARE_LIST", "SOFTWARE_VERSIONS", "SOFTWARE_DEPENDENCIES",
                    "SOFTWARE_VULNERABILITES", "SOFTWARE_SBOM", "UNKNOWN"] = Field(
        ...,
        description="The predicted class label.",
    )

# Define output
class Software(BaseModel):
    name: str
    version: str
    intent: ClassificationResponse

ollama_setup = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
gemini_setup = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/", api_key=os.getenv("GOOGLE_API_KEY"))
client = instructor.from_openai(
    gemini_setup,
    mode=instructor.Mode.JSON,
)


query = "What are the vulnerabilities in aws-sdk-cpp?"
resp = client.chat.completions.create(
    # model="llama3.2",
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    response_model=Software,
)
print(resp.model_dump_json(indent=2))


/Users/iolanigan/github/KHSAAS/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
  "name": "aws-sdk-cpp",
  "version": "unknown",
  "intent": {
    "chain_of_thought": "The question asks about vulnerabilities in a specific software (aws-sdk-cpp).  Therefore, it falls under the SOFTWARE_VULNERABILITES category.",
    "label": "SOFTWARE_VULNERABILITES"
  }
}


In [10]:
queries = [
    "What versions of openai exists?",
    "What are the known vulnerabilities in version 2.2.20 of aws-sdk-cpp?",
    "Generate an SBOM for instructor v1.1.100"
]

def user_intent(query):
    resp = client.chat.completions.create(
    # model="llama3.2",
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    response_model=Software,
    )

    return resp

for query in queries:
    intent = user_intent(query)
    print(intent)

name='openai' version='unknown' intent=ClassificationResponse(chain_of_thought="The question asks for versions of openai.  There is no single, publicly available list of all OpenAI versions. OpenAI releases are not usually versioned in a way that's easily tracked by the public like some other software projects.  They evolve through continuous updates and internal model revisions.", label='SOFTWARE_VERSIONS')
name='aws-sdk-cpp' version='2.2.20' intent=ClassificationResponse(chain_of_thought='The query asks about known vulnerabilities in a specific version of aws-sdk-cpp.  This falls under the category of SOFTWARE_VULNERABILITES.', label='SOFTWARE_VULNERABILITES')
name='instructor' version='v1.1.100' intent=ClassificationResponse(chain_of_thought="The user is requesting an SBOM for the software 'instructor' version 'v1.1.100'.  Therefore, the intent is to generate a Software Bill of Materials.", label='SOFTWARE_SBOM')


In [16]:
from intent import user_intent, handle_intent
res = user_intent(queries[1])
print(f"Name:{res.name}\nVersion:{res.version}\nIntent:{res.intent.label}")
# handle_intent(queries[1])

Name:aws-sdk-cpp
Version:2.2.20
Intent:SOFTWARE_VULNERABILITES


In [17]:
response = user_intent(query)
name = response.name
version = response.version
# print(f"Name:{name}\nVersion:{res.version}\nIntent:{res.intent.label}")

In [3]:
from intent import handle_intent
# query = "What are the vulnerabilities in aws-sdk-cpp?"
query="Generate SBOM for aws-sdk-cpp"
handle_intent(query)

name='aws-sdk-cpp' version='1.0' intent=Classification(chain_of_thought='The input mentions generating an SBOM for aws-sdk-cpp, which directly relates to software bill of materials.  Therefore, the label is SOFTWARE_SBOM.', label='SOFTWARE_SBOM')


MissingSchema: Invalid URL '/api/sbom': No scheme supplied. Perhaps you meant https:///api/sbom?

In [18]:
print(f"Name:{name}\nVersion:{version}\nIntent:{response.intent.label}")

Name:instructor
Version:v1.1.100
Intent:SOFTWARE_SBOM


In [10]:
CHAT_URL = BASE + "/api/chat"
chat = r.post(CHAT_URL, json= {
    "query": queries[0]
})
chat

<Response [500]>